In [1]:
from utils import *
import numpy as np
import pandas as pd
import string
import pickle
#import sys
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

from pymongo import MongoClient

/home/eleanor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/eleanor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/home/eleanor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/eleanor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
client = MongoClient()
db = client.cmv
posts_collection = db.posts
tl_comments_collection = db.tl_comments
deltad_replies_collection = db.deltad_replies

In [3]:
random_state = 42

In [4]:
# establish the set of texts to use (ie posts vs comments)
doctype = 'post'

In [5]:
deltad_docs, undeltad_docs = get_docs(posts_collection, tl_comments_collection,
                                      key_phrase_exists=True)

processing post 0
processing post 50
processing post 100


KeyboardInterrupt: 

In [7]:
train_docs, test_docs, val_docs = doc_split(deltad_docs, undeltad_docs, rand_state=random_state)

In [8]:
len(train_docs)

20057

In [10]:
len(test_docs)

6686

In [11]:
len(val_docs)

6685

In [9]:
train_comment_ids, train_comment_texts, train_labels, train_post_ids, train_post_texts = get_fields(train_docs)
test_comment_ids, test_comment_texts, test_labels, test_post_ids, test_post_texts = get_fields(test_docs)
val_comment_ids, val_comment_texts, val_labels, val_posts_ids, val_post_texts = get_fields(val_docs)

In [ ]:
# pickle train/test/val ids

## Unsupervised Topic Extraction with PyTextRank

In [12]:
# strip unnecessary punctuation from selects 'id' and 'text' fields from docs
train_dicts = get_ptr_dicts(train_docs)

In [26]:
def insert_key_phrases_into_db(list_of_doc_dicts, doctype, collection):
    '''
    Takes in list of doc dictionaries and a doctype ('comment' or 'post'), 
    processes each doc with PyTextRank, obtains key phrases and 
    inserts key phrases into document in Mongodb as 'key_phrases' field.
    '''
    path_stage0 = 'stage0.json'
    path_stage1 = 'stage1.json'
    path_stage2 = 'stage2.json'
    path_stage3 = 'stage3.json'
    
    total_docs = len(list_of_doc_dicts)
    
    failed_ids=[]
    for i, doc_dict in enumerate(list_of_doc_dicts):
        if i % 50 == 0:
            print(f'processing {i} of {total_docs} documents')
        doc_dict['text'] = doc_dict['text'].split('\n_____\n\n')[0]

        try:
            with open(path_stage0, 'w') as f:
                json.dump(doc_dict, f)
            # Stage 1    
            with open(path_stage1, 'w') as f:
                for graf in pytextrank.parse_doc(pytextrank.json_iter(path_stage0)):
                    f.write("%s\n" % pytextrank.pretty_print(graf._asdict()))
                    # print(pytextrank.pretty_print(graf))
            # Stage 2
            graph, ranks = pytextrank.text_rank(path_stage1)
            pytextrank.render_ranks(graph, ranks)
            with open(path_stage2, 'w') as f:
                for rl in pytextrank.normalize_key_phrases(path_stage1, ranks):
                    f.write("%s\n" % pytextrank.pretty_print(rl._asdict()))
                    # to view output in this notebook
                    # print(pytextrank.pretty_print(rl))
            # Stage 3
            kernel = pytextrank.rank_kernel(path_stage2)
            with open(path_stage3, 'w') as f:
                for s in pytextrank.top_sentences(kernel, path_stage1):
                    f.write(pytextrank.pretty_print(s._asdict()))
                    f.write("\n")
                    # to view output in this notebook
                    # print(pytextrank.pretty_print(s._asdict()))
            # Stage 4
            phrase_list = list(set([p for p in pytextrank.limit_keyphrases(path_stage2, phrase_limit=15)]))
            phrases = ", ".join(phrase_list)

            sent_iter = sorted(pytextrank.limit_sentences(path_stage3, word_limit=150), key=lambda x: x[1])
            s = []

            for sent_text, idx in sent_iter:
                s.append(pytextrank.make_sentence(sent_text))

            graf_text = " ".join(s)
            collection.update_one({f'{doctype}_id': {'$eq': doc_dict['id']}},{'$set': {'key_phrases': phrase_list}})
        except:
            failed_ids.append(doc_dict['id'])
            print('failed on ',doc_dict['id'])
            continue

In [24]:
from utils import insert_key_phrases_into_db

In [28]:
insert_key_phrases_into_db(train_dicts[0:2], doctype='comment', collection=tl_comments_collection)

processing 0 of 2 documents


## Vectorize documents using Key Phrases from PyTextRank

In [15]:
post_key_phrases = kparray=get_all_key_phrases(posts_collection, 'post', train_post_ids)
comment_key_phrases = kparray=get_all_key_phrases(tl_comments_collection, 'comment', train_ids)
all_key_phrases = np.union1d(post_key_phrases, comment_key_phrases)
len(all_key_phrases)

79813 unique key phrases from 12122 documents
100999 unique key phrases from 33428 documents


157938

In [ ]:
my_vectorizer = CountVectorizer(vocabulary=all_key_phrases)

In [ ]:
cleaned_train_comments = clean_text(train_comment_texts)
cleaned_test_comments = clean_text(test_comment_texts)
cleaned_val_comments = clean_text(val_comment_texts)

cleaned_train_posts = clean_text(train_post_texts)
cleaned_test_posts = clean_text(test_post_texts)
cleaned_val_posts = clean_text(val_post_texts)

In [ ]:
vectorized_train_comments = my_vectorizer.fit_transform(cleaned_train_comments)

In [ ]:
vectorized_train_comments.

In [ ]:

vectorized_test_comments = my_vectorizer.transform(cleaned_test_comments)